In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import sys
sys.path.insert(0, "../util/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Adding dummies and train-test-split

In [2]:
datingTrainWithDummiesExists = exists('../data/datingTrainWithDummies.csv')
datingTestWithDummiesExists = exists('../data/datingTestWithDummies.csv')
datingTrainWithoutDummiesExists = exists('../data/datingTrainWithoutDummies.csv')
datingTestWithoutDummiesExists = exists('../data/datingTestWithoutDummies.csv')
datingFullWithDummiesExists = exists('../data/datingFullWithDummies.csv')
datingFullWithoutDummiesExists = exists('../data/datingFullWithoutDummies.csv')

relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

In [3]:
with open('../data/columnDataDictionary.json') as d:
    columnDataDictionary = json.load(d)
columnList = columnDataDictionary['columnList']
nonBinaryCategoricalList = columnDataDictionary['nonBinaryCategoricalList']
stringToFloatList = columnDataDictionary['stringToFloatList']
pointDistributionList = columnDataDictionary['pointDistributionList']
partnerList = columnDataDictionary['partnerList']

In [4]:
if (datingTrainWithDummiesExists and datingTestWithDummiesExists and datingFullWithDummiesExists and
    datingTrainWithoutDummiesExists and datingTestWithoutDummiesExists and datingFullWithoutDummiesExists and
    relatedDummiesDictionaryExists):
    datingTrainWithDummies = pd.read_csv('../data/datingTrainWithDummies.csv')
    datingTestWithDummies = pd.read_csv('../data/datingTestWithDummies.csv')
    datingTrainWithoutDummies = pd.read_csv('../data/datingTrainWithoutDummies.csv')
    datingTestWithoutDummies = pd.read_csv('../data/datingTestWithoutDummies.csv')
    datingFullWithDummies = pd.read_csv('../data/datingFullWithDummies.csv')
    datingFullWithoutDummies = pd.read_csv('../data/datingFullWithoutDummies.csv')
    
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    for df in [datingTrainWithDummies,datingTestWithDummies,datingFullWithDummies,datingTrainWithoutDummies,datingTestWithoutDummies,datingFullWithoutDummies]:
        df['zipcode'] = df['zipcode'].apply(str)
        if 'zipcode_o' in list(df.columns):
            df['zipcode_o'] = df['zipcode_o'].apply(str)
        for col in nonBinaryCategoricalList:
            if col in list(df.columns):
                df[col] = df[col].apply(str)
        
else:
    if (datingTrainWithDummiesExists):
        remove('../data/datingTrainWithDummies.csv')
    if (datingTestWithDummiesExists):
        remove('../data/datingTestWithDummies.csv')
    if (datingTrainWithoutDummiesExists):
        remove('../data/datingTrainWithoutDummies.csv')
    if (datingTestWithoutDummiesExists):
        remove('../data/datingTestWithoutDummies.csv')
    if (datingFullWithDummiesExists):
        remove('../data/datingFullWithDummies.csv')
    if (datingFullWithoutDummiesExists):
        remove('../data/datingFullWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    !jupyter nbconvert --execute eda.ipynb
    
    datingData = pd.read_csv('../data/encoded-SpeedDatingData.csv')
    blindDateData = datingData[columnList]
    
    for col in stringToFloatList:
        blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
    blindDateData['zipcode'] = blindDateData['zipcode'].str.replace(',', '')
    for col in nonBinaryCategoricalList:
        blindDateData[col] = blindDateData[col].apply(str)
    
    blindDateCategoricalData = blindDateData.select_dtypes(include=['O'])
    for col in blindDateCategoricalData.columns:
        blindDateData[col]=blindDateData[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in blindDateCategoricalData.columns:
        dummyData = pd.get_dummies(blindDateData[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
            if col in partnerList:
                partnerList.append(str(dummyCol))
        blindDateData = pd.concat([blindDateData,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
        
    partnerList = list(set(partnerList))
    columnDataDictionary = {"columnList": columnList,
                        "nonBinaryCategoricalList": nonBinaryCategoricalList,
                        "stringToFloatList": stringToFloatList,
                        "pointDistributionList": pointDistributionList,
                        "partnerList": partnerList}

    with open('../data/columnDataDictionary.json', 'w') as fp:
            json.dump(columnDataDictionary, fp)
    
    match = datingFullWithDummies['match']
    X = datingFullWithDummies.drop(['match'], axis=1)
    
    datingTrainWithDummies, datingTestWithDummies, matchTrain, matchTest = train_test_split(X, match, test_size=0.2)
    
    datingTrainWithDummies['match'] = matchTrain
    datingTestWithDummies['match'] = matchTest
    
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTestWithDummies.to_csv('../data/datingTestWithDummies.csv',index=False)
    datingFullWithDummies.to_csv('../data/datingFullWithDummies.csv',index=False)
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    datingTrainWithoutDummies = datingTrainWithDummies.drop(dummyColumns, axis=1)
    datingTestWithoutDummies = datingTestWithDummies.drop(dummyColumns, axis=1)
    datingFullWithoutDummies = datingFullWithDummies.drop(dummyColumns, axis=1)
    
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
    datingTestWithoutDummies.to_csv('../data/datingTestWithoutDummies.csv',index=False)
    datingFullWithoutDummies.to_csv('../data/datingFullWithoutDummies.csv',index=False)

Traceback (most recent call last):
  File "/Users/garysimmons/opt/anaconda3/bin/jupyter-nbconvert", line 11, in <module>
    sys.exit(main())
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/jupyter_core/application.py", line 254, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 350, in start
    self.convert_notebooks()
  File "/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 512, in convert_notebooks
    raise ValueError(
ValueError: Please specify an output format with '--to <format>'.
The following formats are available: ['asciidoc', 'custom', 'html', 'latex', 'markdown', 'notebook', 'pdf', 'python', 'rst', 'script', 'slides', 'webp

<ipython-input-4-8301f418b92c>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
<ipython-input-4-8301f418b92c>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blindDateData['zipcode'] = blindDateData['zipcode'].str.replace(',', '')
<ipython-input-4-8301f418b92c>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

## Data Cleaning

### Join partner

In [5]:
for partnerCol in partnerList:
    if (("_o" not in str(partnerCol)) and (partnerCol+"_o" not in datingTrainWithoutDummies.columns) and (partnerCol+"_o" not in datingTrainWithDummies.columns)):
        datingTrainWithDummies = util.joinToPartner(datingTrainWithDummies,maleFullWithDummies)
        datingTrainWithoutDummies = util.joinToPartner(datingTrainWithoutDummies,maleFullWithoutDummies)
        datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
        datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
        break

../util/util.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['iid_o'] = partner_o['iid']
../util/util.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partner_o['pid_o'] = partner_o['pid']


### Get distance

In [ ]:
if (("partnerDistance" not in datingTrainWithoutDummies.columns) or 
    ("partnerDistance" not in datingTrainWithDummies.columns)):
    datingTrainWithoutDummies = util.returnDFWithpartnerDistance(datingTrainWithoutDummies,True)
    datingTrainWithDummies["partnerDistance"] = datingTrainWithoutDummies["partnerDistance"]
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)

13:12:19
0 of 3347: 0.0% complete
0.0% of data is None
13:17:20
100 of 3347: 2.987750224081267% complete
0.05975500448162534% of data is None
13:22:18
199 of 3347: 5.945622945921721% complete
0.11951000896325067% of data is None
13:27:18
299 of 3347: 8.933373170002987% complete
0.268897520167314% of data is None
13:32:19
399 of 3347: 11.921123394084255% complete
0.3884075291305647% of data is None
13:37:19
499 of 3347: 14.908873618165522% complete
0.44816253361219% of data is None
13:42:19
598 of 3347: 17.866746340005975% complete
0.4780400358530027% of data is None
13:47:20
698 of 3347: 20.854496564087242% complete
0.537795040334628% of data is None


### Fix ambiguous scores

In [ ]:
halfwayChangeColumnsFromWithDummies = [str(col) if (("1_s" in str(col)) | ("3_s" in str(col))) for col in datingTrainWithDummies]
halfwayChangeColumnsFromWithoutDummies = [str(col) if (("1_s" in str(col)) | ("3_s" in str(col))) for col in datingTrainWithoutDummies]

if(len(halfwayChangeColumnsFromWithDummies) + len(halfwayChangeColumnsFromWithoutDummies) > 0):
    for question in pointDistributionList:
        questionCols = [str(col) if (question in datingTrainWithDummies)]
        datingTrainWithDummies[f'{question}_sum'] = datingTrainWithDummies[questionCols].sum(axis = 1)
        datingTrainWithoutDummies[f'{question}_sum'] = datingTrainWithoutDummies[questionCols].sum(axis = 1)
        for questionCol in questionCols:
            datingTrainWithDummies[str(questionCol)] = datingTrainWithDummies[str(questionCol)] * 100 / datingTrainWithDummies[f'{question}_sum']
            datingTrainWithoutDummies[str(questionCol)] = datingTrainWithoutDummies[str(questionCol)] * 100 / datingTrainWithoutDummies[f'{question}_sum']

### Replace Nans

# Training

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Individual Testing

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Ensemble Testing